In [1]:
import lcvr_learning as lcl
import numpy as np
import pyvisa
import pandas as pd
rm = pyvisa.ResourceManager('@py')

In [2]:
lcvrs = lcl.lcvr_learning(0x6a,0x6b)

In [3]:
lcvrs.set_ch1_volts(8.97)

In [4]:
lcvrs.outputs_off()

In [3]:
data = lcvrs.get_training_data(20)

Starting training data scan. Don't touch anything please


NameError: name 'pd' is not defined

In [6]:
np.linspace(0,19.9999,5)

array([ 0.      ,  4.999975,  9.99995 , 14.999925, 19.9999  ])